In [4]:
using Plots, BenchmarkTools
using LinearAlgebra
using NAJ

In [44]:
function spectral_radius(A::Matrix{T}) where T<:Number
    eigs = eigvals(A)
    return findmax([abs(x) for x in eigs])[1]
end

function optimal_omega(A::Matrix{<:Real})
    D = Diagonal(A)
    U = UpperTriangular(A)
    L = LowerTriangular(A)
    T = - inv(D+L) * U
    return 1/(1+sqrt(1-spectral_radius(T)))
end

optimal_omega (generic function with 1 method)

In [5]:
A = [2 1; 2 -2]
B = [2 4; 1 -3]
println(spectral_radius(A+B), ", ", spectral_radius(A) + spectral_radius(B))
#spectral_radius(A)

6.437171043518958, 6.151051861499603


In [6]:
spectral_radius(B)

3.7015621187164243

In [7]:
eigvals(A)

2-element Vector{Float64}:
 -2.449489742783178
  2.4494897427831788

In [35]:
function iteration_jacobi3(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector; 
    etol::Number = 1.0e-10,
    Maxiter::Integer = 100_000)
    @assert size(A)[1] == size(A)[2] == size(b)[1]
    @assert Maxiter > 3
    x = zero(x0)
    
    for niter in 1:Maxiter
        @inbounds for i in 1:length(x0)
            @inbounds for j in 1:length(x0)
                if i ≠ j
                    x[i] += -A[i,j] * x0[j]
                end
            end
            @inbounds x[i] = (x[i]+b[i])/A[i, i] 
        end
        if norm(x .- x0, Inf) / norm(x, Inf) < etol
            break
        else 
            x0 = x[:]
            x = zero(x0)

        end
    end
    return x
end

function iteration_seidel3(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector; 
    etol::Number = 1.0e-10, 
    Maxiter = 100_000)
    @assert size(A)[1] == size(A)[2] == size(b)[1]
    
    x = zero(x0)
    for niter in 1:Maxiter
        @inbounds for i in 1:length(x0)
            @inbounds for j in 1:length(x0)
                if j < i
                    x[i] += - A[i, j]*x[j]
                elseif j>i
                    x[i] += - A[i, j]*x0[j] 
                end
            end
            @inbounds x[i] = (x[i]+b[i])/A[i, i] 
        end
        if norm(x .- x0, Inf) / norm(x, Inf) < etol
            break
        else 
            x0 = x[:]
            x = zero(x0)

        end
    end
    return x
end

function iteration_sor3(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector,
    ω::Real; 
    etol::Number = 1.0e-10, 
    Maxiter = 100_000)

    @assert 0 < ω < 2
    
    x = zero(x0)
    for nitter in 1:Maxiter
        for i ∈ 1:length(x0)
            for j ∈ 1:length(x0)
                if j < i
                    x[i] += -A[i, j] *x[j]
                elseif j > i
                    x[i] += -A[i, j] *x0[j]
                end
            end
            x[i] = (1-ω)*x0[i] + 1/A[i, i] * (ω*b[i] + ω *  x[i])
        end
        if norm(x .- x0, Inf)/norm(x, Inf)< etol    
            nitter = i
            return x
        else 
            x0 = x[:]
            x = zero(x)
        end
    end
    return x
end

function iteration_steepest(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector;
    etol::Number = 1.0e-5, 
    Maxiter = 100_000)

    x = similar(x0)
    for i in 1:Maxiter
        v = b - A*x0
        t = dot(v,(b-A*x0))/dot(v, (A*v))
        x = x0 + t*v
        if norm(A*x-b, Inf)<etol
            nitter = i
            println(nitter)
            return x
        else 
            x0 = x
        end
    end
    return nothing
end


function iteration_orthogonal(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector;
    etol::Number = 1.0e-5, 
    Maxiter = 100_000)

    x = similar(x0)
    for i in 1:Maxiter
        v = b - A*x0
        t = dot(v,(b-A*x0))/dot(v, (A*v))
        x = x0 + t*v
        if norm(A*x-b, Inf)<etol
            nitter = i
            println(nitter)
            return x
        else 
            x0 = x
        end
    end
    return nothing
end

iteration_orthogonal (generic function with 1 method)

In [31]:
A = [10.0 -1.0 2.0 0.0; -1 11 -1 3; 2 -1 10 -1; 0 3 -1 8]
b = [6.0, 25, -11, 15]
x0 = [1.0, 1, -1, 1]
A

4×4 Matrix{Float64}:
 10.0  -1.0   2.0   0.0
 -1.0  11.0  -1.0   3.0
  2.0  -1.0  10.0  -1.0
  0.0   3.0  -1.0   8.0

In [48]:
x1 = iteration_jacobi(A, b, x0, etol=1.0e-10)
x12 = iteration_jacobi3(A, b, x0, etol=1.0e-10)
x2 = iteration_seidel3(A, b, x0, etol=1.0e-10)
x3 = iteration_sor3(A, b, x0, 0.7, etol=1.0e-10)
# x4 = iteration_steepest(A, b, x0, etol = 1.0e-5)

# [norm(b-A*x) for x in [x1, x2, x3, x4]]

4-element Vector{Float64}:
  0.9999999998732451
  1.9999999998598545
 -0.9999999999235769
  1.0000000001243183

In [46]:
@btime iteration_jacobi3(A, b, x0, etol=1.0e-10)
@btime iteration_seidel3(A, b, x0, etol=1.0e-10)
@btime iteration_sor3(A, b, x0, 0.6, etol=1.0e-10)

  3.474 μs (80 allocations: 7.50 KiB)
  1.645 μs (32 allocations: 3.00 KiB)
  5.711 μs (107 allocations: 10.03 KiB)


4-element Vector{Float64}:
  0.9999999998482731
  1.9999999998061628
 -0.9999999999038084
  1.0000000001843592

In [51]:
norm(A*x1 - b, Inf)

5.756106702392572e-10

In [52]:
x1

4-element Vector{Float64}:
  0.9999999999779753
  2.000000000036613
 -1.0000000000284026
  1.0000000000408134

In [53]:
x2

4-element Vector{Float64}:
  1.0000000000016507
  2.000000000000011
 -1.0000000000005111
  0.9999999999999318

In [54]:
x3

4-element Vector{Float64}:
  0.9999999998732451
  1.9999999998598545
 -0.9999999999235769
  1.0000000001243183

In [ ]:
spectral_radius(inv(D)*(L+U))

In [ ]:
spectral_radius(inv(D+L)U)

In [ ]:
x'*x

In [ ]:
dot(x,x)

In [41]:
diagonal(A)

UndefVarError: UndefVarError: `diagonal` not defined

In [43]:
UpperTriangular(A)

4×4 UpperTriangular{Float64, Matrix{Float64}}:
 10.0  -1.0   2.0   0.0
   ⋅   11.0  -1.0   3.0
   ⋅     ⋅   10.0  -1.0
   ⋅     ⋅     ⋅    8.0